# Demo of Data-Driven Decks with Drawio

In [1]:
import base64, html, urllib.parse, pathlib as P, tempfile, asyncio, IPython, json
import IPython.display as D, jinja2, babel.support as B
import PyPDF2, traitlets as T, ipywidgets as W, tornado.ioloop, lxml.etree as E
from io import BytesIO
import urllib.parse
from PIL import Image
from nbconvert.filters.markdown_mistune import markdown2html_mistune
from tornado.concurrent import run_on_executor
from concurrent.futures import ThreadPoolExecutor
import requests, requests_cache

In [2]:
requests_cache.install_cache("ddddd", allowable_methods=["GET", "POST"])

## Slides

In [3]:
class Slides(W.HTML):
    """ some number of slides, as PDF
    """
    pdf = T.Unicode()
    error = T.Unicode()
    
    def __init__(self, *args, **kwargs):
        if not kwargs.get("layout"):
            kwargs["layout"] = dict(
                display = "flex",
                flex_flow = "column wrap",
                flex = "1",
                height = "100%",
            )
        super().__init__(*args, **kwargs)
    
    @T.observe("pdf", "error")
    def _on_pdf(self, change):
        if self.error:
            self.value = f"<code>{self.error}</code>"
        elif self.pdf:
            url = f"data:application/pdf;base64,{self.pdf}"
            self.value = f"""
                <a href="{url}">Preview</a>
                <iframe 
                    src="{url}" 
                    style="border: 0; min-width: 400px; min-height: 400px; width: 100%; height: 100%;">
                </iframe>
            """

### DrawioSlides

### Before we begin

> ## This is **NOT READY** for prime-time
> Start the **demo** export server. It will try to install its dependencies with `jlpm`, and requires `nodejs`.
> ```bash
> !python scripts/drawio_export_demo.py
> ```

In [4]:
class DrawioSlides(Slides):
    """ Slides built with drawio-export from drawio XML
    """
    executor = ThreadPoolExecutor(max_workers=1)
    ipynb = T.Unicode()
    xml = T.Union([T.Unicode(), T.Bytes()])
    params = T.Dict()
    url = T.Unicode(default_value="http://localhost:8000")
    
    CORE_PARAMS = dict(
        format="pdf",
        base64="1"
    )
    
    @run_on_executor
    def update_pdf(self):
        # this really needs to be a queue
        self.error = ""
        self.pdf = ""
        self.value = "<blockquote>rendering...</blockqoute>"
        if isinstance(self.xml, str):
            xml = self.xml
        elif isinstance(self.xml, bytes):
            xml = base64.b64encode(self.xml).decode("utf-8")
        try:
            r = requests.post(
                self.url, 
                timeout=None,
                data=dict(
                    xml=xml, 
                    **self.params, 
                    **self.CORE_PARAMS
                ))
            if r.status_code != 200:
                self.error = r.text
            else:
                self.pdf = r.text
        except Exception as err:
            self.error = str(err)
            
    @T.observe("ipynb")
    def _on_ipynb(self, change=None):
        self.error = ""
        try:
            self.xml = json.loads(self.ipynb)["metadata"]["jupyterlab-drawio"]["xml"]
        except Exception as err:
            self.error = str(err)
       
    
    @T.observe("xml")
    def _on_xml(self, change=None):
        tornado.ioloop.IOLoop.current().add_callback(self.update_pdf)

In [5]:
HOW_IT_WORKS = P.Path("testfiles/How it works.dio")
how_it_works = DrawioSlides(xml=HOW_IT_WORKS.read_text())
how_it_works

DrawioSlides(value='', layout=Layout(display='flex', flex='1', flex_flow='column wrap', height='100%'))

Other formats, like `.dio.ipynb` can also be rendered.

In [6]:
DIAGRAM_NOTEBOOK = P.Path("Diagram Notebook.dio.ipynb")
diagram_notebook = DrawioSlides(ipynb=DIAGRAM_NOTEBOOK.read_text())
diagram_notebook

DrawioSlides(value='', layout=Layout(display='flex', flex='1', flex_flow='column wrap', height='100%'))

In [7]:
DIAGRAM_SVG = P.Path("testfiles/A.dio.svg")
diagram_svg = DrawioSlides(xml=DIAGRAM_SVG.read_text())
diagram_svg

DrawioSlides(value='', layout=Layout(display='flex', flex='1', flex_flow='column wrap', height='100%'))

In [8]:
DIAGRAM_PNG = P.Path("testfiles/B.dio.png")
diagram_png = DrawioSlides(xml=DIAGRAM_PNG.read_bytes())
diagram_png

DrawioSlides(value='', layout=Layout(display='flex', flex='1', flex_flow='column wrap', height='100%'))

#### TemplatedDrawioSlides

In [9]:
class TemplatedDrawioSlides(DrawioSlides):
    template = T.Unicode()
    context = T.Dict()
    
    AMP = "&"
    _AMP_ = "_____AMP_____"

    @T.observe("context", "template")
    def _on_context(self, change):
        env = jinja2.Environment(
          extensions=['jinja2.ext.i18n', 'jinja2.ext.autoescape'],
          autoescape=jinja2.select_autoescape(['html', 'xml'])
        )

        self.xml = self.smudge(env.from_string(self.clean(self.template)).render(**{
            key: self.markdown(value)
            for key, value in (self.context or {}).items()
        }))
    
    def clean(self, txt):
        return txt.replace(self.AMP, self._AMP_)

    def smudge(self, txt):
        return txt.replace(self._AMP_, self.AMP)
    
    def markdown(self, md):
        return markdown2html_mistune(md).replace(self.AMP, self._AMP_)

In [10]:
TEMPLATE = P.Path("testfiles/template deck.dio") 
title = TemplatedDrawioSlides(template=TEMPLATE.read_text(), context=dict(
    hero="<h1>???</h1>",
    title="_No title here yet..._"
))
title

TemplatedDrawioSlides(value='', layout=Layout(display='flex', flex='1', flex_flow='column wrap', height='100%'…

In [11]:
logo = """<img src="https://upload.wikimedia.org/wikipedia/commons/3/38/Jupyter_logo.svg" width="200" height="200"></image>"""

In [12]:
title.context = {
    "title": "The _title_ can contain `Markdown`",
    "hero": logo
}

In [13]:
ideas = []
for i in range(4):
    ideas += [
        TemplatedDrawioSlides(
            template=TEMPLATE.read_text(), 
            context=dict(
                title=f"# Idea {i + 1}",
                abstract=f"This is idea {i + 1}. It's better than [idea {i}](#idea-{i})",
                hero=(logo + "\n\n") * (1 + 1) 
            ))
    ]

## Deck

### Deck MK1 Prototype

The simplest deck prototype is just a box.

In [14]:
deck = W.HBox([title, how_it_works, *ideas, diagram_notebook, diagram_png, diagram_svg], 
              layout=dict(display="flex", flex_flow="row wrap"))
# deck

### Deck MK2 Prototype

This builds a composite deck.

In [22]:
class Deck(W.HBox):
    composite = T.Unicode()
    preview = T.Instance(Slides)
    
    def __init__(self, *args, **kwargs):
        if "layout" not in kwargs:
            kwargs["layout"] = dict(display="flex", flex_flow="row wrap")
        super().__init__(*args, **kwargs)
    
    @T.default("preview")
    def _default_preview(self):
        slides = Slides()
        T.dlink((self, "composite"), (slides, "pdf"))
        self.update_composite()
        return slides
    
    @T.observe("children")
    def _on_children(self, change):
        self.update_composite()
    
    def extract_diagrams(self, child):
        if isinstance(child.xml, str):
            node = E.fromstring(child.xml)
        elif isinstance(child.xml, bytes):
            img = Image.open(BytesIO(child.xml))
            node = E.fromstring(urllib.parse.unquote(img.info["mxfile"]))
        
        tag = node.tag
        
        if tag == "mxfile":
            for diagram in node.xpath("//diagram"):
                yield diagram
        elif tag == "mxGraphModel":
            diagram = E.Element("diagram")
            diagram.append(node)
            yield diagram
        elif tag == "{http://www.w3.org/2000/svg}svg":
            diagrams = E.fromstring(node.attrib["content"]).xpath("//diagram")
            for diagram in diagrams:
                yield diagram
    
    def update_composite(self):
        tree = E.fromstring("""<mxfile version="13.3.6"></mxfile>""")
        with tempfile.TemporaryDirectory() as td:
            tdp = P.Path(td)
            merger = PyPDF2.PdfFileMerger()
            for i, child in enumerate(self.children):
                for diagram in self.extract_diagrams(child):
                    tree.append(diagram)
                next_pdf = (tdp / f"doc-{i}.pdf")
                wrote = next_pdf.write_bytes(base64.b64decode(child.pdf.encode("utf-8")))
                if wrote:
                    merger.append(PyPDF2.PdfFileReader(str(next_pdf)))
            output_pdf = tdp / "output.pdf"
            final_pdf = tdp / "final.pdf"
            merger.write(str(output_pdf))
            self.composite_xml = E.tostring(tree).decode("utf-8")
            final = PyPDF2.PdfFileWriter()
            final.appendPagesFromReader(PyPDF2.PdfFileReader(str(output_pdf), "rb"))
            final.addAttachment("drawing.drawio", self.composite_xml.encode("utf-8"))
            with final_pdf.open("wb") as fpt:
                final.write(fpt)
            self.composite = base64.b64encode(final_pdf.read_bytes()).decode("utf-8")

In [26]:
new_deck = Deck(deck.children)

In [27]:
new_deck.preview

Slides(value='\n                <a href="data:application/pdf;base64,JVBERi0xLjMKMSAwIG9iago8PAovVHlwZSAvUGFnZ…